<a href="https://colab.research.google.com/github/kridtapon/AroonFrama-Fusion/blob/main/AroonFrama_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate the Fractal Adaptive Moving Average (FRAMA)
def calculate_frama(df, period=15):
    """
    Calculate Fractal Adaptive Moving Average (FRAMA).
    """
    hl = df['High'] - df['Low']
    hl_rolling = hl.rolling(window=period).sum()

    s = (df['Close'] - df['Close'].shift(period)).abs().rolling(window=period).sum()
    frama = df['Close'].ewm(span=period).mean() + (hl_rolling / s)

    return frama

# Function to calculate Aroon Indicator
def calculate_aroon(df, period=14):
    """
    Calculate Aroon Indicator (Aroon Up and Aroon Down).
    """
    aroon_up = ((df['Close'].rolling(window=period).apply(lambda x: np.argmax(x) + 1) / period) * 100)
    aroon_down = ((df['Close'].rolling(window=period).apply(lambda x: np.argmin(x) + 1) / period) * 100)

    return aroon_up, aroon_down

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Fractal Adaptive Moving Average (FRAMA)
df['FRAMA'] = calculate_frama(df, period=15)

# Calculate Aroon Indicator (Aroon Up and Aroon Down)
df['Aroon_Up'], df['Aroon_Down'] = calculate_aroon(df, period=14)

# Define Entry and Exit signals based on FRAMA and Aroon Indicator
df['Entry'] = (df['FRAMA'] > df['FRAMA'].shift(1)) & (df['Aroon_Up'] > df['Aroon_Down'])
df['Exit'] = (df['FRAMA'] < df['FRAMA'].shift(1)) & (df['Aroon_Down'] > df['Aroon_Up'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           115703.730063
Total Return [%]                         15.70373
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       7197.544032
Max Drawdown [%]                        67.520207
Max Drawdown Duration                      1148.0
Total Trades                                   40
Total Closed Trades                            40
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 30.0
Best Trade [%]                          40.381999
Worst Trade [%]                        -14.137964
Avg Winning Trade [%]                   15.142006
Avg Losing Trade [%]                     -5.00703
